Use this file as a template to copy and build notebooks

In [ ]:
#@title Mount drive and load libraries
import os, pandas as pd, numpy as np
from google.colab import drive

drive.mount('/content/drive/')
path = '/content/drive/MyDrive/msc-project-mbalmf01'
os.chdir(path)

df = pd.read_csv('/content/drive/MyDrive/msc-project-mbalmf01/all_paired/230811_human_paired_seqs_MAPT.csv')